In [ ]:
import torch

In [ ]:
! pip install tokenizer datasets

### Train the tokenizer

In [ ]:
# For the user's convenience `tokenizers` provides some very high-level classes encapsulating
# the overall pipeline for various well-known tokenization algorithm. 
# Everything described below can be replaced by the ByteLevelBPETokenizer class. 

from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.models import BPE
from tokenizers.normalizers import Lowercase, NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel

# First we create an empty Byte-Pair Encoding model (i.e. not trained model)
_tokenizer = Tokenizer(BPE())

# Our tokenizer also needs a pre-tokenizer responsible for converting the input to a ByteLevel representation.
_tokenizer.pre_tokenizer = ByteLevel()

# And finally, let's plug a decoder so we can recover from a tokenized input to the original one
_tokenizer.decoder = ByteLevelDecoder()

In [ ]:
from tokenizers.trainers import BpeTrainer

# We initialize our trainer, giving him the details about the vocabulary we want to generate
trainer = BpeTrainer(vocab_size=1000, show_progress=True, initial_alphabet=ByteLevel.alphabet(), special_tokens=[
            "<s>",
            "<pad>",
            "</s>",
            "<unk>",
            "<mask>"
        ])


_tokenizer.train(files=["tokenized_tunes.txt"], trainer=trainer)



In [ ]:
# saving the tokenized data in our specified folder 
import os
save_path = 'tokenized_data'

In [ ]:
if not os.path.exists(save_path):
    os.makedirs(save_path)
_tokenizer.model.save(save_path)

In [ ]:
_tokenizer

In [ ]:
#_tokenizer.enable_truncation(max_length=512)

In [ ]:
_tokenizer

### Convert the tokenizer 

In [ ]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(save_path)

In [ ]:
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})

In [ ]:
tokenizer.convert_ids_to_tokens(tokenizer.encode(dataset['train']['text'][0]))

### Load Dataset

In [44]:
from datasets import load_dataset, GenerateMode

dataset = load_dataset('text', data_files='2021_07_20_tokenized_tunes.txt', download_mode = GenerateMode.FORCE_REDOWNLOAD)


Using custom data configuration default-742d7de7d4e3a378


Dataset text downloaded and prepared to /Users/jckelly/.cache/huggingface/datasets/text/default-742d7de7d4e3a378/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


In [45]:
dataset['train']['text']

['"R: polka',
 'M: 2/4',
 'K: Gmajor',
 '|: d > g f e | d B A G | E/ F/ G E/ F/ G | B A G F | d > g f e | d B A G | E/ F/ G E/ F/ G | B A G2 :||: e > f g a | b a g f | e/ f/ g e/ f/ g | d/ c/ B/ A/ B A | e > f g a | b a g f | e/ f/ g e/ f/ g | a g g2 :|',
 '"',
 '"R: slip jig',
 'M: 9/8',
 'K: Gmajor',
 'd B B B2 A B A G | d B B B c d e f g | d B B B2 A B A G | e A A d B G A2 e | d B B B2 A B A G | d B B B c d e f g | d B B B2 A B A G | e A A d B G A2 A | B A B g2 e f e d | B A B e/ f/ g e f2 A | B A B g2 e f e d | e A A A B d e2 A | B A B g2 e f e d | B A B e/ f/ g e f2 f | d e f g f e f e d | e A A A B d e f g |',
 '"',
 '"R: strathspey',
 'M: 4/4',
 'K: Gmajor',
 '|: G > A B > G c > A B > G | E < E A > G F < D D2 | G > A B > G c > A B > G | E > A F > D G2 G2 :||: D > D D > D (3 D D D D > D | d > B A > B G < E E2 | D > D D > D (3 D D D D > D | d > B A > B G2 G2 :|',
 '"',
 '"R: reel',
 'M: 4/4',
 'K: Bminor',
 'e A A B e A B B | e A A B g e d B | e A A B e A B B | G2 A B g e d B :| e

In [ ]:
datasets = dataset['train'].train_test_split(0.2)

In [ ]:
tokenized_dataset = datasets.map(lambda x: tokenizer(x['text']), batched=True, num_proc=4, remove_columns=["text"])

In [ ]:
block_size = 128
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=100,
    num_proc=4,
)

### Configure Model

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, max_length = 1024)

In [ ]:
# I chose the hyperparameters here to get an architecture that
# had roughly the same number of paramters as FolkRNN. But I changed a lot of 
# things and IDK what the trade offs are.
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2Config
config = GPT2Config(
    vocab_size = tokenizer.vocab_size,
    n_positions = 512, 
    n_embd = 384,
    n_ctx = 512,
    n_layer = 6,
    n_inner = 6,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)
model = AutoModelForCausalLM.from_config(config)

In [ ]:
model.num_parameters()

### Configure Training

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='./results',          # output directory
  num_train_epochs=10,              # total # of training epochs
  per_device_train_batch_size=16,  # batch size per device during training
  per_device_eval_batch_size=16,   # batch size for evaluation
  evaluation_strategy = "epoch",
  load_best_model_at_end = True
  logging_dir='./logs',            # directory for storing logs
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=lm_datasets['train'],
    eval_dataset=lm_datasets['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [39]:
def generate(start_text = "a b "):
    # encoding the input text
    input_ids = tokenizer.encode(start_text, return_tensors='pt')
    # getting out output
    beam_output = model.generate(
      input_ids,
      do_sample=True,
      max_length = 128,
      top_k=50,
      temperature = 0.3,
      num_return_sequences=5,
      beams=5, 
    
    )
    return beam_output

In [40]:
output = generate()
for i in range(0, 5):
    print(tokenizer.decode(output[i]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


a b a2g2f2g2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2d2f2f2f2f2f2f2f2f2f2f2d2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f2f
a b g2f2|g2f2e2|f2e2d2|e2d2c2|e2d2c2|B2A2B2|A2G2B2|A2G2B2|A2G2B2|A2G2B2|A2G2B2|A2G2B2|A2G2B2|A2G2B2|A2G2B2|A2G2B2|A2G2B2|A2G2B2|A
a b a2a2g2a2g2f2f2f2g2f2f2f2d2f2d2f2f2B2A2A2B2f2f2f2f2d2f2f2f2f2e2f2d2c2d2d2c2d2B2d2e2f2d2d2c2B2d2c2A2B2d2d2B2c2d2B2c2B2d2d2c2A2B
a b a3/2g/2f/2f/2e/2f3/2e/2f/2e/2d3/2A3/2B/2c3/2A3/2B3/2c3/2B3/2c3/2B3/2c3/2B3/2c3/2B3/2c3/2B3/2c3/2B3/2c3/2B3/2B3/2B3/2c3/2B3/2c
a b |g2f2d2|e2d2c2|B2A2G2|B2A2G2|B2A2G2|B2A2G2|G2A2G2|G2G2G2|B2A2G2|G2F2G2|G2G2|A2G2G2|G2G2|G2G2G2|G2G2G2|G2G2|G2G2G2|G2G2|G2G2G2


In [17]:
output

tensor([ 69, 225,  96,  75,  23,  74,  23,  73,  96,  74,  23,  73,  23,  96,
         74,  23,  74,  23,  96,  73,  23,  74,  23,  74,  96,  73,  23,  72,
         23,  96,  73,  22,  74,  23,  73,  22,  96,  74,  23,  69,  23,  96,
         74,  22,  73,  23,  96,  72,  23,  73,  23,  74,  96,  74,  23,  75,
         23,  96,  74,  26,  96,  73,  23,  69,  23,  74,  96,  75,  23,  69,
         23,  69,  96,  74,  23,  72,  23,  73,  96,  72,  23,  71,  23,  96,
         38,  23,  37,  23,  96,  38,  22,  37,  23,  37,  22,  96,  38,  23,
         71,  23,  73,  96,  73,  23,  73,  23,  72,  96,  71,  23,  38,  23,
         96,  37,  23,  38,  23,  37,  96,  38,  23,  72,  23,  71,  96,  38,
         23,  38])

In [28]:
tokenizer.decode(generate())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


TypeError: 'list' object cannot be interpreted as an integer

### TODO / things to try:
#### Pre-processing:
    - regenerate dataset with the proper key, mode, and type
    - fix tokenization issues
    - possibly shift to the same scale 
#### Tokenization:
    - try byte-pair-encoding with key / mode / type
    - try getting simple spaced based tokenization to work
    - look at other archtitectures for domain specific special tokens
    - if we can't get good results, data process + train manually to use our own function 
#### Data-processing:
    - find a better way to create training labels
    - tune batch_size and block_size
    - figure out how padding works
#### Model
    - tune architecture parameters
    - try alternative architectures (ProphetNet?)
#### Training
    - tune training parameters
#### Generation
    - figure out what all of the parameters in generate do 
    - write our own generation code?
#### Evaluation
    - look into quality metrics from existing research
    - write some code to detect training data plagiarism (generation spitting back an input)
    - generate audio from our output for subjective evaluation
    - write some code to check for structural integrity ?
#### Misc
    - figure out a good workflow to train and share model versions 
    

In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('checkpoint')
model = AutoModelForCausalLM.from_pretrained('checkpoint')

In [27]:
tokenizer

PreTrainedTokenizerFast(name_or_path='checkpoint', vocab_size=1000, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'})

In [ ]:
### retrain the model with 
### add the metadata

## figure out how to best generate n tunes / figure out the randomness


